In [4]:
from py_ecc.bn128 import G1, G2, multiply, add, curve_order, Z1, pairing, neg, final_exponentiate, FQ12, is_inf, eq
import galois 
GF = galois.GF(curve_order)

In [12]:
# simple scale demo
a = 10
A = multiply(G1, 10)
print(A)
# (4444740815889402603535294170722302758225367627362056425101568584910268024244, 10537263096529483164618820017164668921386457028564663708352735080900270541420)

# given A, DLOG says you can't find "a" 

# test random pairing
test = pairing(
    multiply(G2,1),
    multiply(G1, 2)
)

print(test)
# G12: (15012325621026644184045762978738412474442356583800543712126643990481298861813, 4267645508599519784673821560899175235932113169305747217206520518895142237067, 9605640887275982760085549877660272063759559624642361043769680838843663043844, 8527741798530570213140864223260614549490569205971543935479128986436935408273, 3170521800353213563435582240280934964129520522016748396998905395639804700884, 15282487952147349684932496442330344938300434905305660692548652058262012064788, 16971187067415765975504500253678755801792873843756283646571921697442136261102, 2076609033601584171799205032777989933146098390599741370266542789167810974568, 9956083666476842555094238955126296965718717241344979949561504280208798934772, 16599042560360271747366000606825634311946267556726426502310580624294554695499, 4754180242998611194659901576356075436720659925499469126121262195274010096072, 8506116078619111915017588317196675174068043321310924689396647807061593583064)
# given ^ G12 point, pairing says we can't find the G1, G2 points that produce it


# but we can verify G12 points by giving the same scalars

# pairing requires powers e(a * b) = e()^ab
# let lhs = (10, 5)... let rhs = (50,1)
# 10 * 5 = 50...... 50 * 1 = 50

lhs = pairing(
    multiply(G2,10),
    multiply(G1, 5)
)

rhs = pairing(
    multiply(G2,50),
    multiply(G1,1)
)

assert eq(lhs,rhs), "not equal"

(4444740815889402603535294170722302758225367627362056425101568584910268024244, 10537263096529483164618820017164668921386457028564663708352735080900270541420)
(15012325621026644184045762978738412474442356583800543712126643990481298861813, 4267645508599519784673821560899175235932113169305747217206520518895142237067, 9605640887275982760085549877660272063759559624642361043769680838843663043844, 8527741798530570213140864223260614549490569205971543935479128986436935408273, 3170521800353213563435582240280934964129520522016748396998905395639804700884, 15282487952147349684932496442330344938300434905305660692548652058262012064788, 16971187067415765975504500253678755801792873843756283646571921697442136261102, 2076609033601584171799205032777989933146098390599741370266542789167810974568, 9956083666476842555094238955126296965718717241344979949561504280208798934772, 16599042560360271747366000606825634311946267556726426502310580624294554695499, 475418024299861119465990157635607543672065992549946912612

In [9]:
# This script demonstrates evaluating a polynomial at a random number is the same as 
# we see that evaluating a polyomial at tau is the same as computing the inner product of the poly coefficients and the powers of tau
tau = GF(20)


# unencrypted evaluation
# Create polynomial 4x^4 + 10x^3 + 7x^2 + 7x + 2
# evaluate at tau = 20 i.e. 4(20)^4 + 10(20)^3 + 7(20)^2 + 7(20) + 2 = 722942

poly = galois.Poly([4,2,3,1], field=GF) * galois.Poly([1,2], field=GF)
# poly = 4x^4 + 10x^3 + 7x^2 + 7x + 2

# Evaluate the poly at tau.
unencrypted_p_tau = poly(tau)
# = 722942

unencrypted_p_tau_G1 = multiply(G1, int(unencrypted_p_tau))
# = 722942 * G1




# encrypted evaluation
# generate powers of tau (1 for each coeff) for 4x^4 + 10x^3 + 7x^2 + 7x + 2
# i.e. [τ^4 G1],[τ^3 G1],[τ^2 G1],[τ^1 G1],[τ^0 G1]
# inner product powers of tau with poly coeffs
# i.e. 4 * [τ^4 G1] + 10 * [τ^3 G1] + 7 * [τ^2 G1] + 7 * [τ^1 G1] + 2 * [τ^0 G1]
# result should be equal to 

def generate_powers_of_tau(tau, degree):
    """Computes and prints the powers of tau up to the specified degree and returns them
    as elliptic curve points by multiplying each power by the base point G1."""
    powers_of_tau = []
    for i in range(degree+1):
        powers_of_tau.append(multiply(G1, int(tau**i)))
    return powers_of_tau

# create [τ^0 G1],[τ^1 G1][τ^2 G1]...[τ^degree +1 G1]
powers_of_tau = generate_powers_of_tau(tau, poly.degree)

def inner_product(powers_of_tau, coeffs):
    """Computes the elliptic curve point representing the inner product of the reversed poly coefficients
    and the powers of tau points."""
    accumulator = Z1
    for tau_power, coeff in zip(powers_of_tau, coeffs):
        term = multiply(tau_power, int(coeff))
        accumulator = add(accumulator, term)
    return accumulator

encrypted_p_tau_G1 = inner_product(powers_of_tau, poly.coeffs[::-1])

# test that the encrypted = unencrypted
assert eq(encrypted_p_tau_G1, unencrypted_p_tau_G1), "doesn't match"


